In [1]:
!pip install ipynb sklearn progressbar2

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from ipynb.fs.full.utils import *
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
res = 24

In [4]:
def draw(imgs, labels):
    fig, axs = plt.subplots(1, len(imgs), figsize=(15, 3), sharey=True, subplot_kw=dict(projection='3d'))
    for i in range(len(imgs)):
        cube1 = (imgs[i][:,:,:] >= 1)
        axs[i].voxels(cube1, facecolors="blue")
        axs[i].set_title(labels[i])
    plt.show()

In [5]:
class Histogram:
    def __init__(self, histX, histY, histZ, label):
        self.histX = histX
        self.histY = histY
        self.histZ = histZ
        self.label = label
        
    @classmethod
    def load(self, filename):
        with open(filename + '.pkl', 'rb') as input:
            return pickle.load(input)
    
    def save(self, filename):
        with open(filename + '.pkl', 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
            
    def draw(self):
        fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharey=True)
        axs[0].bar(range(res), self.histX)
        axs[0].set_title(self.label + ' X')
        axs[1].bar(range(res), self.histY)
        axs[1].set_title('Y')
        axs[2].bar(range(res), self.histZ)
        axs[2].set_title('Z')
        plt.show()

In [6]:
def calculate_histogram(obj, label, res):
    objy = np.moveaxis(obj, [0, 1, 2], [1, 0, 2])
    objz = np.moveaxis(obj, [0, 1, 2], [1, 2, 0])
    #calculate the histogram in 3 axis
    histx = np.zeros(res, dtype=int)
    histy = np.zeros(res)
    histz = np.zeros(res)
    for i in range(res):
        ox = obj[i].reshape([res * res])
        histx[i] = ox[ox == 1].size

        oy = objy[i].reshape([res * res])
        histy[i] = oy[oy == 1].size

        oz = objz[i].reshape([res * res])
        histz[i] = oz[oz == 1].size
    return Histogram(histx, histy, histz, label)


In [7]:
def draw_histogram(histograms):
    fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharey=True)
    axs[0].bar(range(res), histograms.histX)
    axs[0].set_title(label + ' X')
    axs[1].bar(range(res), histograms.histY)
    axs[1].set_title('Y')
    axs[2].bar(range(res), histograms.histZ)
    axs[2].set_title('Z')
    plt.show()

# Calculate and save histograms

In [8]:
bar2 = progressbar.ProgressBar(max_value=len(all_labels)-1)

for i, label in enumerate(all_labels):
    bar2.update(i)
    training, training_labels = load_data(res, [label], 'train', 1, 0, False)
    for j in range(len(training)):
        histograms = calculate_histogram(training[j], label, res)
        histograms.save('histograms/train-' + histograms.label)
    test, testlabels = load_data(res, [label], 'test', 1, False, False)
    for j in range(len(test)):
        histograms = calculate_histogram(test[j], label, res)
        histograms.save('histograms/test-' + histograms.label)


100% (39 of 39) |########################| Elapsed Time: 0:00:19 ETA:  00:00:00

# Prepare Network 

In [93]:
training, training_labels = load_data(res, all_labels, 'train', 1, 0, False)
test, test_labels = load_data(res, all_labels, 'test', 1, 0, False)

In [112]:
class Model:
    def __init__(self, res, num_classes, verbosity):
        self.prepare_model(res, num_classes, verbosity)
        
    def prepare_model(self, res, num_classes, verbosity = 0):
        pass
    
    def train(self, training, training_labels, res, num_classes, verbosity = 0, log = False, epochs = None):
        log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        if verbosity == 1 and log:
            print("Log file " + log_dir)
        if log:
            tensorboard_callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)]
        else:
            tensorboard_callbacks = None

        validation_split = 0.1
        if epochs == None:
            epochs = 2 * num_classes

        self.model.fit(
            prepare_data(training, num_classes, res), 
            training_labels, 
            epochs=epochs,
            verbose=verbosity,
            callbacks=tensorboard_callbacks,
            validation_split=validation_split
    )
    
    def evaluate(self, test, test_labels, verbose):
        return self.model.evaluate(
            test,
            test_labels, 
            verbose=2
        )

In [113]:
class ModelConv3D(Model):

    def __init__(self, res, num_classes, verbosity):
        super().__init__(res, num_classes, verbosity)
    
    def prepare_model(res, num_classes, verbosity = 0):
        self.model = keras.Sequential([
            keras.layers.Conv3D(
                    res, 
                    kernel_size=(6),
                    strides=(2),
                    activation='relu', 
                    kernel_initializer='he_uniform', 
                    data_format="channels_last",
                    input_shape=(res, res, res, 1)
                ),        
            keras.layers.Conv3D(
                    res, 
                    kernel_size=(5),
                    strides=(2),
                    activation='relu', 
                    kernel_initializer='he_uniform'
                ),        
            #keras.layers.Conv3D(
            #        res, 
            #        kernel_size=(2),
            #        activation='relu', 
            #        kernel_initializer='he_uniform'
            #    ),        

            #keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(res/2, activation='relu'),
            keras.layers.Dense(num_classes, activation='softmax')
        ])

        learning_rate = 0.0001

        self.model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        if (verbosity == 1):
            print(model.summary())

    def prepare_data(data, num_classes, res):
        return data.reshape(len(data), res, res, res, 1)
    model_type = 'conv3d'


In [114]:
class ModelDense(Model): 
    
    def __init__(self, res, num_classes, verbosity):
        super().__init__(res, num_classes, verbosity)

    def prepare_model(self, res, num_classes, verbosity = 0):
        self.model = keras.Sequential([
            keras.layers.Flatten(input_shape=(res, res, res)),
            keras.layers.Dense(res*2, activation='relu'),
            keras.layers.Dense(res/2, activation='sigmoid'),
            keras.layers.Dropout(.2, input_shape=(2,)),
            keras.layers.Dense(40, activation='softmax')
        ])

        learning_rate = 0.0001

        self.model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        if (verbosity == 1):
            print(self.model.summary())
            
    def prepare_data(self, data, num_classes, res):
        return data
    model_type = 'dense'

In [116]:
num_classes = len(all_labels)
model = ModelDense(res, num_classes, 1)
model.train(training, training_labels, res, num_classes, 2, False, 10)


Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_37 (Flatten)         (None, 13824)             0         
_________________________________________________________________
dense_111 (Dense)            (None, 48)                663600    
_________________________________________________________________
dense_112 (Dense)            (None, 12)                588       
_________________________________________________________________
dropout_37 (Dropout)         (None, 12)                0         
_________________________________________________________________
dense_113 (Dense)            (None, 40)                520       
Total params: 664,708
Trainable params: 664,708
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
276/276 - 2s - loss: 3.4584 - accuracy: 0.0949 - val_loss: 3.8418 - val_accuracy: 0.0204
Epoch 2/

In [117]:
test_loss, test_acc = model.evaluate(
    model.prepare_data(test, num_classes, res),  
    test_labels, 
    verbose=2
)


76/76 - 0s - loss: 2.8710 - accuracy: 0.3777
